<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模型
假設每個輸入項xi不再只是單一數值，而是一個由xil,...,xik所組成的向量k。  
多元遞歸模型架設如下：


```
yi = α+β₁χɪʟ+...+βkχɪʟ
```
在多元迴歸模型中，參數向量通常以β來表示。  
若把常數項也包含進來，只需多加一個欄位。
* beta = [alpha, beta_1 , ... ,beta_k]
* x_i = [1, x_i1 , ... ,x_ik]
  




於是我們的模型就變成：  
[常數項，朋友數量，每天工作時數，有博士學位與否]

In [69]:
from typing import List

Vector = List[float]

def dot(v:Vector,w:Vector)->float:
  #計算v_1*w_1+... +v_n*w_n
  assert len(v)==len(w),"兩個向量必須有相同的維度"

  return sum(v_i*w_i for v_i,w_i in zip(v,w))

In [70]:
def predict (x:Vector,beta:Vector) -> float:
  """假設每個x_i的第一個元素都是1"""
  return dot(x,beta)

# 關於最小平方模型的進一步假設

1. x的每個元素必須是**線性獨立**的：每個元素皆無法透過其他元素加總來得到。  
若該假設不成立，則無法得估計出β。
2. x中的每個元素全都與誤差項ε無關。  
若該假設不成立，估計β時會發生系統性錯誤。

# 套入模型
目標：找到某組beta係數值，讓誤差平方得到最小化的結果

## 誤差函數

In [71]:
from typing import List

def error(x:Vector,y:float,beta:Vector) -> float:
  return predict(x,beta) - y

def squared_error(x:Vector,y:float,beta:Vector) -> float:
  return error(x,y,beta)**2

進行測試

In [72]:
x = [1,2,3]
y = 30
beta = [4,4,4] #預測值為 4+8+12 = 24

assert error(x,y,beta) == -6
assert squared_error(x,y,beta) == 36

## 計算梯度
透過微積分

In [73]:
def squerror_gradient(x:Vector,y:float,beta:Vector) -> float:
  err = error(x,y,beta)
  return [2*err*x_i for x_i in x]

In [74]:
assert squerror_gradient(x,y,beta) == [-12,-24,-36]

## 1. 梯度遞減 (書本上作法)
用來找出最佳的beta值

In [75]:
import random
import tqdm

from typing import List

Vector = List[float]

def add( v:Vector, w:Vector) -> Vector:
  assert len(v) == len(w) ,"兩個向量必須有相同的維度"

  return [ v_i+w_i for v_i,w_i in zip(v,w)]

def vector_sum(vectors:List[Vector]) -> Vector:
  #先檢查vertors這個向量列表是否為空
  assert vectors,"列表中沒有向量!"

  #檢查vertors 向量列表內的所有向量都具有相同的維度
  num_elements=len(vectors[0])
  assert all(len(v)==num_elements for v in vectors),"向量維度不一致"

  #所有vectors[i]相加起來，是結果的第i個元素值
  return [sum(vector[i] for vector in vectors) for i in range(num_elements)]


def scalar_multiply(c:float,v:Vector) -> Vector:
  return [c*v_i for v_i in v]

def vector_mean(vectors:List[Vector])->Vector:
  n=len(vectors)
  return scalar_multiply(1/n,vector_sum(vectors))

def gradient_step(v:Vector,gradient:Vector,step_size:float) -> Vector:
  """從v沿著gradient的方向移動step_size的距離"""
  assert len(v) == len(gradient)
  step = scalar_multiply(step_size,gradient)
  return add(v,step)

In [76]:
def least_squares_fit(
          xs:List[Vector],
          ys:List[float],
          learning_rate:float=0.001,
          num_steps:int=1000,
          batch_size:int=1)->Vector:
  """
  找出最小化平方誤差和的beta值
  假設模型y = dot(x,beta)
  """
  # 一開始先使隨機方式做出猜測
  guess = [random.random() for _ in xs[0]]

  for _ in tqdm.trange(num_steps,desc="least squares fit"):
    for start in range(0,len(xs),batch_size):
      batch_xs = xs[start:start+batch_size]
      batch_ys = ys[start:start+batch_size]

      gradient = vector_mean([squerror_gradient(x,y,guess) for x,y in zip(batch_xs,batch_ys)])

      guess = gradient_step(guess,gradient,-learning_rate)
  
  return guess

## 套用到資料中

In [77]:
inputs = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

In [78]:
import numpy as np

random.seed(0)

# 嘗試使用錯誤的方式選擇num_iters和step_size
# 這會需要花一點時間
learning_rate = 0.001

beta = least_squares_fit(inputs,daily_minutes_good,learning_rate ,5000,25)

assert 30.50 < beta[0] < 30.70 #常數
assert 0.96 < beta[1] < 1.00 #朋友的數量
assert -1.89 < beta[2] < -1.85 #每天的工作時數
assert 0.91 < beta[3] < 0.94 #有博士學問的話

np.round(np.array(beta),3)

least squares fit: 100%|██████████| 5000/5000 [00:02<00:00, 1781.28it/s]


array([30.515,  0.975, -1.851,  0.914])

在實務上不會使用梯度遞減的方式來計算線性回歸的方法，  

## ✪ [線性代數求解：最小平方近似](https://ithelp.ithome.com.tw/articles/10186400) (網路資源)
![公式解](http://i.imgur.com/EngTecm.png)  
利用numpy函式庫來協助運算


In [79]:
import numpy as np

def least_square_approximation(vector_x,vector_y):
  x = np.array(vector_x)
  y = np.array(vector_y)

  # A = (X^T*X)⁻¹
  transpose_x = np.transpose(x)
  temp = transpose_x @ x
  A = np.linalg.inv(temp)

  # D = X^T * y
  D = transpose_x @ y

  # β = A*D = 最終解 = (X^T*X)⁻¹ * X^T * y
  β = A @ D
  return list(np.round(β,3)) #將最終的解,取小數點後三位

In [80]:
least_square_approximation(inputs,daily_minutes_good)

[30.579, 0.973, -1.865, 0.923]

## 運算後，得到的方程式是：
```
minutes = 30.579 + 0.973 friends - 1.865 woek hours + 0.923 phd
```


# 解釋模型
　　
**所有其他因素皆相等的情況下**：
* 每額外增加一個朋友，每天花在網站上的時間就會增加將近1分鐘。
* 使用者的工作時間美額外增加１小時，每天花在網站上的時間就會減少將近２分鐘。
* 每個擁有博士學位的使用者，每天都會在網站上多待近1分鐘。
  
這些變數彼此間的交互關係，對於朋友多或少的人來說工作時數的影響**可能是不相同**的。  

# 套入優度
利用**R平方**的值來觀察：

In [81]:
def mean(xs:List[float]) -> float:
  return sum(xs) / len(xs)

def de_mean(xs:List[float]) -> List[float]:
  x_bar = mean(xs)
  return [x - x_bar for x in xs] 


In [82]:
def sum_of_sqerrors(x , y , beta):
    return sum( error(x_i , y_i , beta) ** 2
               for x_i, y_i in zip(x, y))

def total_sum_squares(y:Vector) ->float:
  """每個y_i與平均值之間的差值的總平方和，即「總變異量」"""
  return sum(v**2 for v in de_mean(y))

def mulitiple_r_squared(xs:List[float],ys:Vector,beta:Vector) ->float:
  """
  模型掌握到y變異量的比率，即(1-模型未掌握到y變異量的比率)
  """
  sum_of_squared_errors = sum(error(x,y,beta)**2 for x , y in zip(xs,ys))
  return 1.0 - (sum_of_squared_errors/total_sum_squares(ys))

def total_sum_of_squares(y):
    """the total squared variation of y_i's from their mean"""
    return sum(v ** 2 for v in de_mean(y))

In [83]:
rsq= mulitiple_r_squared(inputs,daily_minutes_good,beta)
assert 0.67 < rsq < 0.68
print("%.3f"%rsq)

0.680


在多元迴歸模型中，還要觀察各個係數的標準差，以衡量所得到每個βi估計值。  
  
**衡量誤差的假設**：
誤差項εi是一個平均值為0，標準差σ的**獨立常態隨機變數**。  
模型中係數的標準差越大，該係數的可靠程度就越低。  

# Bootstrap (重複取樣)
假設我們有一組其中包含N個資料的樣本，是根據某種未知的分布所生成的：

```
data = get_sample(num_points = n
```
如果重複取得多組的樣本組，我們可以計算出許多樣本組的中位數，進而觀察這些中位數的分布情況。  
這種情況下，可以「重複採樣(bootstrap)」從原本的資料集內，重新選出n個資料重新組合成新的資料集以取代原本的。


In [84]:
from typing import TypeVar , Callable

X = TypeVar('X')
Stat = TypeVar('Stat')

def bootstrap_sample(data:List[X]) -> List[X]:
  """以隨機的方式重複取樣 len(data)"""
  return [random.choice(data) for _ in data]

def bootstrap_statistic(data:List[X],stats_fn:Callable[[List[X]],Stat],num_samples:int) -> List[X]:
  """ 從 data 取出 num_samples 個重複取樣的樣本，然後進行 states_fn 的計算 """
  return [stats_fn(bootstrap_sample(data)) for _ in range(num_samples)]


考慮下列兩個資料集：

In [85]:
#101個資料集，其值全都非常接近100
close_to_100 = [99.5 + random.random() for _ in range(101)]

#101個資料集，其中50個值很接近0，另外50個值很接近200
far_from_100 = ([99.5 + random.random()]+[random.random() for _ in range(50)]+[200 + random.random() for _ in range(50)])

計算這組資料集的中位數，結果都會相當靠近100。

In [86]:
def sum_of_squares(v:Vector) -> float:
  return dot(v,v)

def _medium_odd(xs:List[float]) ->float:
  return sorted(xs)[len(xs)//2]

def _medium_even(xs:List[float]) ->float:
  sorted_xs=sorted(xs)
  hi_midpoint=len(xs)//2
  return (sorted_xs[hi_midpoint-1]+sorted_xs[hi_midpoint])/2

def median(v:List[float])->float:
  return _medium_even(v) if len(v)%2 == 0 else _medium_odd(v)

In [87]:
import math

def variance(xs:List[float]) ->float:
  assert len(xs),"至少有兩個元素才能計算變異數"

  n = len(xs)
  deviations = de_mean(xs)
  return sum_of_squares(deviations) / (n-1)

def standard_deviation(xs:List[float]) ->float:
  return math.sqrt(variance(xs))

In [88]:
# 數值都很靠近100
medians_close = bootstrap_statistic( close_to_100 , median ,100)

In [89]:
# 數值很接近0，亦有都多很靠近200
medians_far = bootstrap_statistic( far_from_100 , median ,100)

* 第一組資料中位數的標準差會很接近0
* 第二組資料中位數的標準差會很接近100

In [90]:
assert standard_deviation(medians_close) < 1
assert standard_deviation(medians_far) > 90

## 迴歸係數的標準差
採用相同的方法，估算出迴歸係數的標準差。  
針對相同資料集不斷進行重複取樣，就能估算出不同的beta值。  
* 某自變數經多次取樣後仍沒太大變動 → 該係數的估計值應該十分可靠
* 某自變數經多次取樣後的變化很大 → 該係數的估計值就不那麼可靠

In [91]:
from typing import Tuple

import datetime

def estimate_sample_beta(pairs:List[Tuple[Vector,float]]):
  learning_rate = 0.001
  x_sample = [x for x , _ in pairs]
  y_sample = [y for _ , y in pairs]
  beta = least_square_approximation(x_sample,y_sample)
  print("bootstrap sample",beta)
  return beta

In [92]:
bootstrap_betas = bootstrap_statistic(list(zip(inputs,daily_minutes_good)),estimate_sample_beta,100)

bootstrap sample [30.045, 0.931, -1.912, 1.215]
bootstrap sample [31.798, 1.003, -2.131, 1.901]
bootstrap sample [29.218, 1.028, -1.752, 1.135]
bootstrap sample [29.793, 0.969, -1.822, 2.099]
bootstrap sample [30.641, 1.025, -1.955, 0.636]
bootstrap sample [31.363, 0.909, -1.807, 0.241]
bootstrap sample [29.767, 0.993, -1.826, 1.285]
bootstrap sample [29.744, 0.967, -1.787, 0.979]
bootstrap sample [29.842, 1.017, -1.986, 2.491]
bootstrap sample [32.779, 0.876, -1.836, 0.529]
bootstrap sample [29.886, 0.931, -1.737, 0.928]
bootstrap sample [31.166, 1.037, -2.137, 1.354]
bootstrap sample [30.897, 0.964, -1.845, -0.723]
bootstrap sample [29.698, 1.0, -1.778, 2.444]
bootstrap sample [30.359, 0.994, -1.946, 1.932]
bootstrap sample [30.999, 1.035, -2.14, 1.119]
bootstrap sample [29.922, 1.005, -1.836, 0.993]
bootstrap sample [30.898, 0.888, -1.641, 0.847]
bootstrap sample [31.189, 0.872, -1.631, 0.399]
bootstrap sample [30.822, 0.91, -1.916, 0.973]
bootstrap sample [29.61, 1.136, -1.768, 2.0

In [93]:
bootstarp_standard_errors = [
  standard_deviation([beta[i] for beta in bootstrap_betas])
  for i in range(4)]

print(bootstarp_standard_errors)

[1.1682705603246244, 0.08243581831591254, 0.11094275710263993, 1.0043213234354185]


我們可以用這些標準差的值，來檢定每個βi係數是不是「等於零」。  
並提出以下假設：
* 零假設(H₀)：**βi=0**
* 對立假設(H₁)：**βi≠0**
  
定義一個統計量如下：  
![公式](https://lh3.googleusercontent.com/3c2Ut_B-HmaR4KyErS_m2LOTSacVpzPjShCHhYokeYI-nJXWjXyn3NXMMKiFC8FbWXhqNtLtNAQrE9Y4KaAVK1poTrd_RVgHW8JfvDPANAz_8j8oG1ZFd44KKLdWU8e3RkUXRPHGhv-eFc3XrRQgCW2wOssYfGSwqz56FHIKJFgczzwntTcdVVMYV8LAENyHcrANIxO88FYWwas55tmtudxz1qRS_W5mOqATnmHRETImjEtla9Pc-vjiRDcF4NSXfZL6aamh8abg0VIQ2VK7tBv6MrYhPiJk6MLEKUJaAMSkkA0AFuB7k80Q3xqSdnu-UYOGqG5id3QnPsVzu_vEBF5yli9wfSMgfPPs29EsYGl-mwd-rR5IoisBJDmHYbPR7vfS8mdxpN1Wx4nYnP_eypfri4nqblpgPIH717f3oBkor0n_YBJ7uwUIgxJ_UifLMsjYX4GrIN-H-YE_ovh9OxIKiXykdZALZrs8XH73Yc2sj3OjRaUBy9GaUJHYAihpFP5r3R5xLK0388Ur1nwIM0YjWLfvR9Xhu-8Vgc8zk_0Ff-p6Y2vypJi9CE3YC5bfGRuQSe_s71ZDSG_XSR_t6tuLZEgK-1Km2gDXvzkFsv5zghGwNvnTehBqaSYBSDD0GPj4SKpGxvuYlzyG_50fpgQzf9ZtefoEAr1GaHhdlqqpY3wxPnRZqMYg8yJRfQ=w175-h99-no?authuser=0)  
該統計量代表我們對βj的統計值，與其相應標準差估計值兩者相除的結果，其本身應該會形成一個「自由度為n-k」的學生t分佈。  
如果我們知道**學生t分布的累積分佈函數**，就可以針對最小平方法所得出的每個細數計算出相對應的p值。
>p值代表 零假設成立的情況下，觀察到該估計值的可能性有多大。  
  
儘管我們無法得知學生t分布的累積分佈函數；隨著自由度越來越大，該分佈仍會趨近於常態分佈。  
因此，只要n遠大於k。我們就能使用常態累積分佈函數得出結果。

In [94]:
import math
def normal_cdf(x:float,mu:float=0,sigma:float=1)->float:
  return (1+math.erf((x-mu)/math.sqrt(2)/sigma))/2

In [95]:
def p_value(beta_hat_j:float,sigma_hat_j:float)->float:
  if beta_hat_j >0:
    #如果係數為正，我們必須把「看到更大值」的機率乘以兩倍
    return 2*( 1 - normal_cdf(beta_hat_j/sigma_hat_j))
  else:
    #否則就是把「看到更小值」的機率乘以兩倍
    return 2* normal_cdf(beta_hat_j/sigma_hat_j)

In [96]:
assert p_value(30.58,1.27) < 0.001  # 常數項
assert p_value(0.972,0.103) < 0.001  # 朋友數量
assert p_value(-1.865,0.115) < 0.001  # 工作時數
assert p_value(0.923,1.249) > 0.4   # 博士

# 正則化
在誤差項中加入一懲罰項，以避免得到過大的beta係數值。  
例如：**脊回歸(ridge regression)**  
在誤差項中加入一個正比於beta_i平方和的懲罰項(beta_0常數項除外)

In [97]:
# alpha項：一個超參數(hyperparameter)，他控制的是懲罰項的影響程度
# 有時也以lambda代稱，但已經被Python定義為匿名函式了
def ridge_penalty(beta:Vector,alpha:float)->float:
  return alpha * dot(beta[1:],beta[1:])

def squared_error_ridge(x:Vector,y:float,beta:Vector,alpha:float)->float:
  """估計誤差值的平方，再加上對beta的脊懲罰項(ridge_penalty)"""
  return error(x,y,beta)**2 + ridge_penalty(beta,alpha)

接著，引入梯度遞減的作法：

In [98]:
def ridge_penalty_gradient(beta:Vector,alpha:float)->Vector:
  """純粹只估算脊懲罰項(ridge penalty)所對應的梯度值"""
  return [0.]+[2*alpha*beta_j for beta_j in beta[1:]]

def sqerror_ridge_gradient(x:Vector,y:float,beta:Vector,alpha:float)->Vector:
  """
  計算出第i個平方誤差所對應的梯度值，其中也包括脊懲罰項
  """
  return add(squerror_gradient(x,y,beta),ridge_penalty_gradient(beta,alpha))

修改原本的least_squares_fit函式：

In [99]:
def least_squares_fit_ridge(
          xs:List[Vector],
          ys:List[float],
          alpha:float=0.0,
          learning_rate:float=0.001,
          num_steps:int=1000,
          batch_size:int=1)->Vector:
  """
  找出最小化平方誤差和的beta值
  假設模型y = dot(x,beta)
  """
  # 一開始先使隨機方式做出猜測
  guess = [random.random() for _ in xs[0]]

  for _ in tqdm.trange(num_steps,desc="least squares fit"):
    for start in range(0,len(xs),batch_size):
      batch_xs = xs[start:start+batch_size]
      batch_ys = ys[start:start+batch_size]

      gradient = vector_mean([sqerror_ridge_gradient(x,y,guess,alpha) for x,y in zip(batch_xs,batch_ys)])

      guess = gradient_step(guess,gradient,-learning_rate)
  
  return guess

### 假定alpha設定為0

In [116]:
random.seed(0)
learning_rate=0.001
beta_0 = least_squares_fit_ridge(inputs,daily_minutes_good,0.0,learning_rate,5000,25)

#[30.51,0.97,-1.85,0.91]
print("\nbeta_0:",list(np.round(np.array(beta_0),2)))
print("beta_0內積：",dot(beta_0[1:],beta_0[1:]) )
assert 5 < dot(beta_0[1:],beta_0[1:]) < 6
assert 0.67 < mulitiple_r_squared(inputs,daily_minutes_good,beta_0) < 0.69

least squares fit: 100%|██████████| 5000/5000 [00:04<00:00, 1107.92it/s]


beta_0: [30.51, 0.97, -1.85, 0.91]
beta_0內積： 5.21088501552135


## 增加 alpha 值
套入優度會變差，而beta的相應係數值也會變得越來越小。

In [113]:
random.seed(0)
learning_rate=0.001
beta_0_1 = least_squares_fit_ridge(inputs,daily_minutes_good,0.1,learning_rate,5000,25)

#[30.8,0.95,-1.83,0.54]
print("\nbeta_0_1:",list(np.round(np.array(beta_0_1),2)))
print("beta_0_1內積：",dot(beta_0_1[1:],beta_0_1[1:]) )
assert 4 < dot(beta_0_1[1:],beta_0_1[1:]) < 5
assert 0.67 < mulitiple_r_squared(inputs,daily_minutes_good,beta_0_1) < 0.69

least squares fit: 100%|██████████| 5000/5000 [00:04<00:00, 1109.77it/s]


beta_0_1: [30.8, 0.95, -1.83, 0.54]
beta_0_1內積： 4.554201608410774


In [114]:
random.seed(0)
learning_rate=0.001
beta_1 = least_squares_fit_ridge(inputs,daily_minutes_good,1.0,learning_rate,5000,25)

#[30.65,0.9,-1.68,0.1]
print("\nbeta_1:",list(np.round(np.array(beta_1),2)))
print("beta_1內積：",dot(beta_1[1:],beta_1[1:]) )
assert 3 < dot(beta_1[1:],beta_1[1:]) < 4
assert 0.67 < mulitiple_r_squared(inputs,daily_minutes_good,beta_1) < 0.69

least squares fit: 100%|██████████| 5000/5000 [00:04<00:00, 1102.09it/s]


beta_1: [30.65, 0.9, -1.68, 0.1]
beta_1內積： 3.6274483410516174


In [115]:
random.seed(0)
learning_rate=0.001
beta_10 = least_squares_fit_ridge(inputs,daily_minutes_good,10,learning_rate,5000,25)

#[28.31, 0.67, -0.9, -0.01]
print("\nbeta_10:",list(np.round(np.array(beta_10),2)))
print("beta_10內積：",dot(beta_10[1:],beta_10[1:]) )
assert 1 < dot(beta_10[1:],beta_10[1:]) < 2
assert 0.5 < mulitiple_r_squared(inputs,daily_minutes_good,beta_10) < 0.6

least squares fit: 100%|██████████| 5000/5000 [00:04<00:00, 1112.63it/s]


beta_10: [28.31, 0.67, -0.9, -0.01]
beta_10內積： 1.2706657437961764


## 結論
「PhD」係數隨著懲罰增加而消失，故可推論該係數很有可能根本就等於零。  
與先前推估的結果是一致的。

# lasso迴歸
傾向強迫某些係數變為零的作法，但不適合搭配梯度遞減的做法。